# [과제1] 교제 Part4 작물 잎 사진으로 질병 분류하기

## 개요

- 이미지 분류 모델을 활용하여 작물 잎 사진의 종류와 질병 유무를 분류
- Transfer Learning, Convolutional Neural Network
- 총 데이터 수: 40,000개

## 데이터 설명

- Apple_Apple_scab
- Apple_Black_rot
- Apple_Cedar_apple
- Apple_healthy: ---질병 없는 사과
- Cherry_Powdery
- Cherry_healthy: **질병 없는 체리**
- Corn_Cercospora
- Corn_Common_rust
- Corn_Northern_Leaf
- Corn_healthy: **질병 없는 옥수수**
- Grape_Black_rot
- Grape_Esca
- Grape_Leaf_blight
- Grape_healthy: **질병 없는 포도**
- Peach_Bacterial_spot
- Peach_healthy: **질병 없는 복숭아**
- Pepper_Bacterial_spot
- Pepper_bell_healthy: **질병 없는 후추**
- Potato_Early_blight
- Potato_Late_blight
- Potato_healthy: **질병 없는 감자**
- Strawberry_healthy: **질병 없는 딸기**
- Strawberry_scorch
- Tomato_Bacterial_spot
- Tomato_Early_blight
- Tomato_Late_blight
- Tomato_Leaf_Mold
- Tomato_Septoria_leaf
- Tomato_Spider_mites
- Tomato_Target_Spot
- Tomato_mosaic_virus
- Tomato_Yellow_Leaf
- Tomato_healthy: **질병 없는 토마토**

## 데이터 분할을 위한 폴더 생성

In [1]:
import os
import shutil

# 원본 데이터셋이 위치한 경로
original_dataset_dir = '.datasets'
# os.listdir() 메서드는 해당 경로 하위에 있는 모든 폴더의 목록을 가져옴
classes_list = os.listdir(original_dataset_dir)

# 나눈 데이터를 저장할 폴더를 생성
base_dir = '.splitted'
os.mkdir(base_dir)

# 분리 후에 각 데이터를 저장할 하위 폴더 train, val, test를 생성
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# train, validation, test 폴더 하위에 각각 클래스 목록 폴더를 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))
    os.mkdir(os.path.join(test_dir, clss))

## 데이터 분할과 클래스별 데이터 수 확인

In [2]:
import math

# for 문을 통해 모든 클래스에 대한 작업을 반복
for clss in classes_list:
    # path 위치에 존재하는 모든 이미지 파일의 목록을 변수 fnames에 저장
    path = os.path.join(original_dataset_dir, clss)
    fnames = os.listdir(path)

    # Train, Validation, Test 데이터의 비율을 지정(6:2:2)
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    # Train 데이터에 해당하는 파일의 이름을 train_fname에 저장
    train_fnames = fnames[:train_size]
    print('Train size(', clss, '):', len(train_fnames))
    # 모든 Train 데이터에 대해 for문의 내용을 반복
    for fname in train_fnames:
        # 복사할 원본 파일의 경로
        src = os.path.join(path, fname)
        # 복사한 후 저장할 파일의 경로
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        # src의 경로에 해당하는 파일을 dst 경로에 저장
        shutil.copyfile(src, dst)

    # Validation 데이터에 대해 위의 코드 반복
    validation_fnames = fnames[train_size:(train_size + validation_size)]
    print('Validation size(', clss, '):', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)

    # Test 데이터에 대해 위의 코드 반복
    test_fnames = fnames[(train_size +
                          validation_size):(train_size + validation_size +
                                            test_size)]
    print('Test size(', clss, '):', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)

Train size( Apple___Apple_scab ): 378
Validation size( Apple___Apple_scab ): 126
Test size( Apple___Apple_scab ): 126
Train size( Apple___Black_rot ): 372
Validation size( Apple___Black_rot ): 124
Test size( Apple___Black_rot ): 124
Train size( Apple___Cedar_apple_rust ): 165
Validation size( Apple___Cedar_apple_rust ): 55
Test size( Apple___Cedar_apple_rust ): 55
Train size( Apple___healthy ): 987
Validation size( Apple___healthy ): 329
Test size( Apple___healthy ): 329
Train size( Cherry___healthy ): 512
Validation size( Cherry___healthy ): 170
Test size( Cherry___healthy ): 170
Train size( Cherry___Powdery_mildew ): 631
Validation size( Cherry___Powdery_mildew ): 210
Test size( Cherry___Powdery_mildew ): 210
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ): 307
Validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ): 102
Test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ): 102
Train size( Corn___Common_rust ): 715
Validation size( Corn___Common_rust ): 238
Test siz

## 베이스라인 모델 학습을 위한 준비

In [3]:
import torch

# 현재 사용 중인 환경에서 GPU를 사용할 수 있는 지 확인
USE_CUDA = torch.cuda.is_available()
# GPU 사용 가능 하면 'cuda' 불가능 하면 'cpu'
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

# 배치 사이즈 지정
BATCH_SIZE = 256
# 에포크 횟수 지정
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# transforms.Compose 메서드는 이미지 데이터의 전처리, Augmentation 등의 과정에서 사용되는 메서드
# transforms.Resize 메서드는 이미지의 크기를 조정
# transforms.ToTensor 메서드는 이미지를 Tensor 형태로 변환하고 모든 값을 0에서 1사이로 정규화
transform_base = transforms.Compose(
    [transforms.Resize((64, 64)),
     transforms.ToTensor()])

# ImageFolder 메서드는 데이터셋을 불러오는 메서드
# root 옵션에 데이터를 불러올 경로를 설정
# transform 옵션에 데이터를 불러온 후 전처리 또는 Augmentation을 할 방법을 지정
train_dataset = ImageFolder(root='.splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='.splitted/val', transform=transform_base)

from torch.utils.data import DataLoader

# DataLoader는 불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리하는 역할을 수행
# shuffle=True로 설정하면 데이터의 순서가 섞여 모델이 학습을 할 떄 Label 정보의 순서를 기억하는 것을 방지
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4)
val_loader = DataLoader(val_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=4)

C:\Users\HP\miniconda3\envs\gpu2.6\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\HP\miniconda3\envs\gpu2.6\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


## 베이스라인 모델 설계

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# 딥러닝 모델과 관련된 기본적인 함수를 포함하는 nn.Module 클래스를 상속하여 사용
class Net(nn.Module):

    # __init__ 함수에서 모델에서 사용할 모든 Layer를 정의
    def __init__(self):
        # nn.Module 내에 있는 메서드를 상속받아 사용
        super(Net, self).__init__()

        # 첫 번째 2d Convolutional Layer 정의
        # 파라미터는 순서대로 채널 수, 출력 채널 수, 커널 크기
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # 2차원 MaxPooling을 실행하는 Layer 정의
        # 파라미터는 순서대로 커널 크기, Stride
        self.pool = nn.MaxPool2d(2, 2)
        # 입력 채널 32, 출력 채널 64, 커널 크기 3인 Convolutional Layer 정의
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # 입력 채널 64, 출력 채널 64, 커널 크기 3인 Convolutional Layer 정의
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

        # Flatten 이후에 사용될 첫 번째 Fully Connected Layer 정의
        self.fc1 = nn.Linear(4096, 512)
        # Flatten 이후에 사용될 두 번째 Fully Connected Layer 정의
        # fc1의 출력 채널 수와 동일하고, 모델에서 마지막 Layer로 사용될 것이기 때문에
        # 출력 채널 수는 분류 클래스의 수와 동일
        self.fc2 = nn.Linear(512, 33)

    def forward(self, x):
        # Convolution 연산을 진행한 후 Feature Map 생성
        x = self.conv1(x)
        # 생성된 Feature Map값에 비선형 활성 함수인 ReLU() 적용
        x = F.relu(x)
        # MaxPooling 적용
        x = self.pool(x)
        # 25%의 노드를 Dropout
        # training=self.training 부분은 학습 모드일 때와 검증 모드일 때 각각 다르게 적용되기 위해 존대
        # 학습 과정에서는 일부 노드를 랜덤하게 제외시키지만, 평가 과정에서는 모든 노드를 사용
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        # 생성된 Feature Map을 1차원으로 펼치는 과정인 Flatten을 수행
        x = x.view(-1, 4096)
        # Flatten된 1차원 Tensor를 fc1에 적용
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 모델의 마지막 Layer
        # 클래스의 개수에 해당하는 33개의 출력값을 가짐
        x = self.fc2(x)

        # 마지막 Layer의 33개의 결과값에 softmax() 함수를 적용하여 데이터가 각 클래스에 속할 확률을 Output값으로 출력
        return F.log_softmax(x, dim=1)


# 정의한 CNN 모델 Net()의 새로운 객체를 생성
# to(DEVICE)를 통해 모델을 현재 사용 중인 장비에 할당
model_base = Net()
model_base = model_base.to(DEVICE)
# optimizer는 Adam으로 설정하고, Learning Rate는 0.001로 설정
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

## 모델 학습을 위한 함수

In [5]:
def train(model, train_loader, optimizer):
    # 입력받는 모델을 학습 모드로 설정
    model.train()
    # train_loader에는 (data, target) 형태가 미니 배치 단위로 묶여 있음
    for batch_idx, (data, target) in enumerate(train_loader):
        # data와 target 변수를 사용 중인 장비에 할당
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 이전 Batch의 Gradient값이 optimizer에 저장되어 있으므로 optimizer를 초기화
        optimizer.zero_grad()
        # 데이터를 모델에 입력하여 Output값을 계산
        output = model(data)
        # 모델에서 계산한 Output 값인 예측값과 Target 값 사이의 Loss를 계산
        # 분류 문제에 적합한 Cross Entropy Loss를 사용
        loss = F.cross_entropy(output, target)
        # 위에서 계산한 Loss 값을 바탕으로 Back Propagation을 통해 계산한 Gradient 값을 각 Parameter에 할당
        loss.backward()
        # 위에서 각 Parameter에 할당된 Gradient 값을 이용해 모델의 Parameter를 업데이트
        optimizer.step()

## 모델 평가를 위한 함수

In [6]:
def evaluate(model, test_loader):
    # 입력받는 모델을 평가 모드로 설정
    model.eval()
    # 미니 배치별로 Loss를 합산해서 저장할 변수인 test_loss를 정의
    test_loss = 0
    # 올바르게 예측한 데이터 수를 세는 변수인 correct를 정의
    correct = 0

    # 모델을 평가하는 단계에서는 모델의 Parameter를 업데이트하지 않아야 함
    # with torch.no_grad() 메서드를 이용하여 해당 부분을 실행하는 동안 모델의 Parameter 업데이트를 중단
    with torch.no_grad():
        # 앞서 학습했던 것과 같이, test_loader에는 (data, target) 형태가 미니 배치단위로 묶여 있음
        for data, target in test_loader:
            # data, target 변수를 사용 중인 장비로 할당
            data, target = data.to(DEVICE), target.to(DEVICE)
            # 데이터를 모델에 입력하여 output 값을 계산
            output = model(data)

            # 모델에서 계산한 output값인 예측값과 target 값 사이의 Loss를 계산
            test_loss = F.cross_entropy(output, target, reduction='sum').item()

            # 모델에 입력된 Test 데이터가 33개의 클래스에 속할 각각의 확률값이 Ouput으로 출력함
            # 이 중 가장 높은 값을 가진 인덱스를 예측값으로 저장
            pred = output.max(1, keepdim=True)[1]
            # target.view_as(pred)를 통해 target Tensor의 구조를 pred Tensor와 같은 모양으로 정렬
            # view_as() 메서드는 적용 대상 Tensor를 메서드에 입력되는 Tensor의 모양대로 재정렬하는 함수
            # view() 함수는 정렬하고 싶은 Tensor의 모양을 숫자로 직접 지정한다는 점에서 차이가 있음
            # eq() 메서드는 객체 간의 비교 연산자로, pred.eq(target.view_as(pred))은 pred와 target.view_as(pred)의 값이 일치하면 1, 일치하지 않으면 0을 반환
            correct += pred.eq(target.view_as(pred)).sum().item()

        # 모든 미니 배치에서 합한 Loss값을 Batch 수로 나누어 미니 배치마다 계산된 Loss 값의 평균을 구함
        test_loss /= len(test_loader.dataset)
        # 모든 미니 배치에서 합한 정확도 값을 Batch 수로 나누어 미니 배치마다 게산된 정확도 값의 평균을 구함
        test_accuracy = 100 * correct / len(test_loader.dataset)

        # 측정한 Test Loss와 정확도를 반환
        return test_loss, test_accuracy

## 모델 학습 실행하기

In [7]:
import time
import copy


def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
    # 정확도가 가장 높은 모델의 정확도를 저장하는 best_acc를 정의
    best_acc = 0.0
    # 정확도가 가장 높은 모델을 저장할 변수 best_model_wts를 선언
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        # 한 Epoch당 소요되는 시간을 측정하기 위해 해당 Epoch을 시작할 때의 시간을 저장
        since = time.time()
        # 앞서 정의한 train() 함수를 이용하여 모델을 학습
        train(model, train_loader, optimizer)
        # 앞서 정의한 evaluate() 함수를 이용하여 해당 Epoch에서의 학습 Loss와 정확도를 계산
        train_loss, train_acc = evaluate(model, train_loader)
        # 앞서 정의한 evaluate() 함수를 이용하여 해당 Epoch에서의 검증 Loss와 정확도를 계산
        val_loss, val_acc = evaluate(model, val_loader)

        # 현재 Epoch의 검증 정확도가 최고 정확도보다 높다면
        # best_acc를 현재 Epoch의 검증 정확도로 업데이트하고,
        # 해당 Epoch의 모델을 best_model_wts에 저장
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        # 한 Epoch당 소요된 시간을 계산
        time_elapsed = time.time() - since
        print('---------- epoch {} ----------'.format(epoch))
        # 해당 Epoch의 학습 Loss와 정확도를 출력
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(
            train_loss, train_acc))
        # 해당 Epoch의 검증 Loss와 정확도를 출력
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        # 한 Epoch당 소요된 시간을 출력
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                    time_elapsed % 60))

    # 최종적으로 정확도가 가장 높은 모델을 불러온 뒤, 반환
    model.load_state_dict(best_model_wts)
    return model


# 앞서 정의한 train_baseline() 함수를 이용하여 Baseline 모델을 학습
base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
# 학습이 완료된 모델을 저장
torch.save(base, 'baseline.pt')

---------- epoch 1 ----------
train Loss: 0.0154, Accuracy: 42.35%
val Loss: 0.0129, Accuracy: 42.20%
Completed in 0m 55s
---------- epoch 2 ----------
train Loss: 0.0097, Accuracy: 61.39%
val Loss: 0.0085, Accuracy: 60.37%
Completed in 0m 24s
---------- epoch 3 ----------
train Loss: 0.0063, Accuracy: 72.28%
val Loss: 0.0054, Accuracy: 71.30%
Completed in 0m 24s
---------- epoch 4 ----------
train Loss: 0.0054, Accuracy: 78.91%
val Loss: 0.0044, Accuracy: 76.86%
Completed in 0m 25s
---------- epoch 5 ----------
train Loss: 0.0044, Accuracy: 83.05%
val Loss: 0.0040, Accuracy: 80.35%
Completed in 0m 25s
---------- epoch 6 ----------
train Loss: 0.0032, Accuracy: 84.95%
val Loss: 0.0051, Accuracy: 82.25%
Completed in 0m 26s
---------- epoch 7 ----------
train Loss: 0.0025, Accuracy: 88.59%
val Loss: 0.0031, Accuracy: 85.84%
Completed in 0m 25s
---------- epoch 8 ----------
train Loss: 0.0030, Accuracy: 88.11%
val Loss: 0.0032, Accuracy: 84.98%
Completed in 0m 25s
---------- epoch 9 -----

## Transfer Learning

- ResNet50 모델을 불러온 후, Fine-Tuning하여 작물 잎을 분류하는 주제에 맞추어 사용

## Transfer Learning을 위한 준비

In [17]:
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize([64, 64]),
        # 이미지를 무작위로 좌우 반전
        transforms.RandomHorizontalFlip(),
        # 이미지를 무작위로 상하 반전
        transforms.RandomVerticalFlip(),
        # 이미지의 일부를 랜덤하게 잘라내어 52x52 사이즈로 변경
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        # 이미지가 Tensor 형태로 전환된 이후에 정규화를 시행
        # 첫 번째 배열은 각각 Red, Green, Blue 채널 값에서 정규화를 적용할 평균값을 의미
        # 두 번째 배열은 각각 Red, Green, Blue 채널 값에서 정규화를 적용할 표준편차 값을 의미
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val':
    transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '.splitted'
image_datasets = {
    x: ImageFolder(root=os.path.join(data_dir, x),
                   transform=data_transforms[x])
    for x in ['train', 'val']
}
dataloaders = {
    x: DataLoader(image_datasets[x],
                  batch_size=BATCH_SIZE,
                  shuffle=True,
                  num_workers=4)
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

## Pre-Trained Model 불러오기

In [18]:
from torchvision import models

# resnet50 모델을 불러옴
# pretrained 옵션을 True로 설정하면 미리 학습된 모델의 Parameter를 그대로 가져옴
resnet = models.resnet50(pretrained=True)
# ResNet50 모델의 마지막 Layer의 출력 채널 수가 33개가 아님
# 따라서 불러온 모델을 이 프로젝트의 맞추고자 모델의 마지막 Fully Connected Layer 대신 출력 채널의 수가 33개인 새로운 Layer를 추가할 것
# 이를 위해 불러온 ResNet50에서 마지막 Layer의 입력 채널의 수를 저장
num_ftrs = resnet.fc.in_features
# 불러온 모델의 마지막 Fully Connected Layer를 새로운 Layer로 교체
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad,
                                 resnet.parameters()),
                          lr=0.001)

from torch.optim import lr_scheduler

# StepLR() 메서드는 Epoch에 따라 Learning Rate를 변경하는 역할을 함
# step_size=7, gamma=0.1로 설정하면 7 Epoch마다 0.1씩 곱해 Learning Rate를 감소시킨다는 의미
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## Pre-Trained Model의 일부 Layer Freeze하기

In [19]:
# 해당 Layer가 몇 번째 Layer인지를 나타내는 변수 ct 정의
ct = 0
# children() 메서드는 모델의 자식 모듈을 반복 가능한 객체로 반환
# 여기서는 resnet 모델의 모든 Layer 정보를 담고 있음.
for child in resnet.children():
    # for 문을 한 번 반복한 후 다음 Layer를 지칭하기 위해 ct값 1증가
    ct += 1
    if ct < 6:
        # ResNet50에 존재하는 10개의 Layer 중에서 1번부터 5번 Layer의 Parameter는 업데이트퇴지 않도록 고정하고,
        # 6번부터 10번 Layer의 Parameter는 학습 과정에서 업데이트하도록 설정
        for param in child.parameters():
            param.requires_grad = False

## Transfer Learning 모델 학습과 검증을 위한 함수

In [20]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # 현재 진행 중인 epoch 출력
        print('----------epoch {}----------'.format(epoch + 1))
        since = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # 모든 데이터의 Loss를 합산해서 저장할 변수인 running_loss를 정의
            running_loss = 0.0
            # 올바르게 예측한 경위의 수를 세는 변수인 running_corrects를 정의
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                # 학습 단계에서만 모델의 Gradient를 업데이트
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate: ', l_r)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                    time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 정확도가 가장 높은 모델을 불러온 후 반환
    model.load_state_dict(best_model_wts)
    return model

## 모델 학습 실행하기

In [21]:
# 앞서 정의한 train_resnet() 함수를 이용하여 Fine-Tuning
model_resnet50 = train_resnet(resnet,
                              criterion,
                              optimizer_ft,
                              exp_lr_scheduler,
                              num_epochs=EPOCH)
# 학습이 완료된 모델을 저장
torch.save(model_resnet50, 'resnet50.pt')

----------epoch 1----------
learning rate:  [0.001]
train Loss: 0.5954 Acc: 0.8186
val Loss: 0.3328 Acc: 0.8919
Completed in 0m 29s
----------epoch 2----------
learning rate:  [0.001]
train Loss: 0.2217 Acc: 0.9302
val Loss: 0.2543 Acc: 0.9199
Completed in 0m 29s
----------epoch 3----------
learning rate:  [0.001]
train Loss: 0.1865 Acc: 0.9410
val Loss: 0.1970 Acc: 0.9367
Completed in 0m 29s
----------epoch 4----------
learning rate:  [0.001]
train Loss: 0.1324 Acc: 0.9557
val Loss: 0.1508 Acc: 0.9546
Completed in 0m 29s
----------epoch 5----------
learning rate:  [0.001]
train Loss: 0.1244 Acc: 0.9596
val Loss: 0.1585 Acc: 0.9513
Completed in 0m 29s
----------epoch 6----------
learning rate:  [0.001]
train Loss: 0.1098 Acc: 0.9647
val Loss: 0.1238 Acc: 0.9601
Completed in 0m 29s
----------epoch 7----------
learning rate:  [0.0001]
train Loss: 0.0815 Acc: 0.9735
val Loss: 0.0926 Acc: 0.9700
Completed in 0m 29s
----------epoch 8----------
learning rate:  [0.0001]
train Loss: 0.0446 Acc

## 베이스라인 모델 평가를 위한 전처리

In [22]:
# 베이스라인 모델의 성능을 평가하기 위해 DataLoader를 생성
# 모델을 학습시킬 때 사용한 학습, 검증 데이터와 동일한 방법으로 전처리를 수행
transform_base = transforms.Compose(
    [transforms.Resize([64, 64]),
     transforms.ToTensor()])
test_base = ImageFolder(root='.splitted/test', transform=transform_base)
test_loader_base = DataLoader(test_base,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=4)

## Transfer Learning 모델 평가를 위한 전처리

In [23]:
# Transfer Learning 모델의 성능 평가를 위해 사용할 테스트 데이터의 DataLoader를 생성
# 모델을 학습시킬 때 사용한 학습, 검증 데이터와 동일한 방법으로 전처리 수행
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_resNet = ImageFolder(root='.splitted/test', transform=transform_resNet)
test_loader_resNet = DataLoader(test_resNet,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=4)

## 베이스라인 모델 성능 평가하기

In [24]:
# 저장했던 베이스라인 모델 불러오기
baseline = torch.load('baseline.pt')
# 모델을 평가 모드로 설정
baseline.eval()
# 테스트 데이터에 대한 정확도를 측정
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc: ', test_accuracy)

baseline test acc:  93.00287895856803


## Transfer Learning 모델 성능 평가하기

In [25]:
# 저장했던 Transfer Learning 모델 불러오기
resnet50 = torch.load('resnet50.pt')
# 모델을 평가 모드로 설정
resnet50.eval()
# 테스트 데이터에 대한 정확도를 측정
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc: ', test_accuracy)

ResNet test acc:  98.96107147327575


미리 학습된 모델을 불러와 일부를 Fine-Tuning하는 것이 더 높은 예측 성능

<br />

Pre-Trained 모델은 약 1,400만 개의 이미지를 학습해놓은 모델이고, 이 모델에는 다양한 이미지의 Feature가 학습되어 있음

<br />

따라서 Pre-Trained Model이 더 좋은 성능을 나타낼 가능성이 높다는 것은 매우 당연한 사실